In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
import os

In [159]:
#Kaggle dataset download
#kaggle datasets download -d bhanupratapbiswas/inventory-analysis-case-study
display(Purchase)

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2015-12-21,2016-01-02,2016-01-04,2016-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-02,2016-01-07,2016-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2015-12-24,2016-01-02,2016-01-09,2016-02-16,21.32,5,106.60,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,49_GARIGILL_22298,49,22298,Zorvino Vyds Sangiovese,750mL,90058,ZORVINO VINEYARDS,13593,2016-12-19,2016-12-28,2017-01-09,2017-02-06,8.06,12,96.72,2
2372470,1_HARDERSFIELD_19556,1,19556,Zorvino Bacca Z Blackberry,750mL,90058,ZORVINO VINEYARDS,13593,2016-12-19,2016-12-27,2017-01-09,2017-02-06,9.39,12,112.68,2
2372471,66_EANVERNESS_22297,66,22297,Zorvino Vyds Pearz,750mL,90058,ZORVINO VINEYARDS,13593,2016-12-19,2016-12-26,2017-01-09,2017-02-06,6.75,12,81.00,2
2372472,69_MOUNTMEND_19557,69,19557,Zorvino Fragole Z Strawberry,750mL,90058,ZORVINO VINEYARDS,13593,2016-12-19,2016-12-26,2017-01-09,2017-02-06,9.39,12,112.68,2


In [153]:
path = 'inventory_analysis_case_study'

Inventory_path = os.path.join(path, 'BegInvFINAL12312016.csv')
Purchase_path = os.path.join(path, 'PurchasesFINAL12312016.csv')
Sales_path = os.path.join(path, 'SalesFINAL12312016.csv')
Invoice_path = os.path.join(path, 'InvoicePurchases12312016.csv')
Prices_path = os.path.join(path, '2017PurchasePricesDec.csv')

Inventory = pd.read_csv(Inventory_path)
Purchases = pd.read_csv(Purchase_path)
Sales = pd.read_csv(Sales_path)
Invoices = pd.read_csv(Invoice_path)
Prices = pd.read_csv(Prices_path)


#print(Sales.info())
Sales['SalesDate'] = pd.to_datetime(Sales['SalesDate'])
Purchases['ReceivingDate'] = pd.to_datetime(Purchases['ReceivingDate'])
Purchases['PODate'] = pd.to_datetime(Purchases['PODate'])
Inventory['startDate'] = pd.to_datetime(Inventory['startDate'])


Purchases = Purchases[Purchases['ReceivingDate'] <= max(Sales['SalesDate'])]


Sales1 = Sales.rename(columns={'SalesDate': 'Date', 'SalesQuantity': 'StockChange'})
Sales1['StockChange'] = -Sales1['StockChange']

Inventory1 = Inventory.rename(columns={'startDate': 'Date', 'onHand': 'StockChange'})

Purchases1 = Purchases.rename(columns={'ReceivingDate': 'Date', 'Quantity': 'StockChange'})

StockChanges = pd.concat([Inventory1[['InventoryId', 'Date', 'StockChange']], Sales1[['InventoryId', 'Date', 'StockChange']], Purchases1[['InventoryId', 'Date', 'StockChange']]])

StockChanges['StockLevel'] = StockChanges.groupby(['InventoryId', 'Date'])['StockChange'].cumsum()
StockLevels = StockChanges[['InventoryId', 'Date', 'StockLevel']].reset_index(drop=True)

StockLevels = StockLevels.merge(Inventory, how='left', on=['InventoryId'])



StockLevels = StockLevels[['InventoryId', 'Date', 'StockLevel', 'Store', 'City', 'Brand', 'Description', 'onHand']]




#display(StockLevels[StockLevels['StockLevel'] == 0])

In [149]:
Demand_temp = Sales.groupby(['InventoryId', 'SalesDate'])['SalesQuantity'].sum().reset_index()

avg_demand_daily = Demand_temp.groupby(['InventoryId'])['SalesQuantity'].mean().to_dict()

x = "10_HORNSEY_1001"
y = avg_demand_daily.get(x)
demand_dist = np.random.poisson(y)
print(demand_dist)

Purchases['DeliveryTime'] = Purchases['ReceivingDate'] - Purchases['PODate']
Purchases['DeliveryTime'] = Purchases['DeliveryTime'].dt.days.astype(int)

delivery_time = Purchases.groupby('InventoryId')['DeliveryTime'].apply(list).to_dict()
print(delivery_time.get(x))

4
{'10_HORNSEY_10030': [12], '10_HORNSEY_10057': [9], '10_HORNSEY_10058': [5], '10_HORNSEY_10062': [5], '10_HORNSEY_10238': [11], '10_HORNSEY_10266': [8], '10_HORNSEY_10283': [7, 11, 4, 10, 11, 10], '10_HORNSEY_10296': [11], '10_HORNSEY_10372': [3], '10_HORNSEY_10420': [10, 8, 8, 9, 6, 8], '10_HORNSEY_10421': [8, 11, 9, 8, 10, 10, 9, 6], '10_HORNSEY_10422': [9], '10_HORNSEY_10443': [5], '10_HORNSEY_10451': [7, 10, 9, 9], '10_HORNSEY_10453': [8], '10_HORNSEY_10469': [7], '10_HORNSEY_10472': [11], '10_HORNSEY_10579': [9], '10_HORNSEY_10596': [6, 8, 5, 8, 7, 9, 9], '10_HORNSEY_106': [6, 7, 10], '10_HORNSEY_10609': [6], '10_HORNSEY_10621': [10], '10_HORNSEY_10715': [8, 8, 6], '10_HORNSEY_10723': [5, 5, 7], '10_HORNSEY_10727': [4], '10_HORNSEY_10874': [9, 8, 10, 9], '10_HORNSEY_10895': [10, 9, 7, 7], '10_HORNSEY_10927': [6], '10_HORNSEY_10931': [11, 8], '10_HORNSEY_1094': [11], '10_HORNSEY_10957': [9, 6], '10_HORNSEY_11029': [6], '10_HORNSEY_11067': [8], '10_HORNSEY_11141': [9, 5, 8], '10_H

In [172]:
def simulate_inventory(inventory_id, days, storage_cost = 0.1):
   demand = np.random.poisson(avg_demand_daily.get(inventory_id))
   leadtime = np.random.choice(delivery_time.get(inventory_id))
   

   return demand, leadtime


simulate_inventory("34_PITMERDEN_5215")





(1, 6)

In [108]:
print(poissson())

[1.41892394]


In [104]:
print(np.random.poisson(5))


#Inventory['Size'] = Inventory['Size'].str.replace(r'\D', '', regex=True).replace("", 0).astype(int)
#display(Inventory)

6


In [63]:
Inventory['Volume'] = Inventory['Size'] * Inventory['onHand'] 
capacity_per_store = Inventory.groupby("Store")['Volume'].sum().reset_index()
capacity_per_store_per_product = Inventory.groupby(["Store", "Brand"])[["Volume", "onHand"]].sum().reset_index()
display(capacity_per_store)


,Store,Volume
0,1,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
1,2,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
2,3,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
3,4,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
4,5,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
...,...,...
74,75,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
75,76,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
76,77,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...
77,78,750mL750mL750mL750mL750mL750mL750mL750mL750mL7...


In [25]:
Inventory_1_58 = Inventory[(Inventory['Brand'] == 58) & (Inventory['Store'] == 1)]
display(Inventory_1_58)
Sales_1_58 = Sales[(Sales['Brand'] == 58) & (Sales['Store'] == 1)]
display(Sales_1_58)
Purchases_1_58 = Purchases[(Purchases['Brand'] == 58) & (Purchases['Store'] == 1)]
display(Purchases_1_58)

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2016-01-01


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
14426,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,1,12.99,12.99,1/7/2016,750,1,0.79,8320,SHAW ROSS INT L IMP LTD
951497,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,2,25.98,12.99,2/10/2016,750,1,1.57,8320,SHAW ROSS INT L IMP LTD
951498,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,1,12.99,12.99,2/11/2016,750,1,0.79,8320,SHAW ROSS INT L IMP LTD
951499,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,1,12.99,12.99,2/13/2016,750,1,0.79,8320,SHAW ROSS INT L IMP LTD
951500,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,1,12.99,12.99,2/14/2016,750,1,0.79,8320,SHAW ROSS INT L IMP LTD
951501,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,1,12.99,12.99,2/16/2016,750,1,0.79,8320,SHAW ROSS INT L IMP LTD
951502,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,1,12.99,12.99,2/18/2016,750,1,0.79,8320,SHAW ROSS INT L IMP LTD
951503,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,2,25.98,12.99,2/20/2016,750,1,1.57,8320,SHAW ROSS INT L IMP LTD
951504,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,1,12.99,12.99,2/23/2016,750,1,0.79,8320,SHAW ROSS INT L IMP LTD


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
6575,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8166,2015-12-23,2016-01-01,2016-01-08,2016-02-11,9.28,6,55.68,1
267751,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8835,2016-02-08,2016-02-17,2016-02-27,2016-03-31,9.28,6,55.68,1
307840,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8920,2016-02-14,2016-02-26,2016-03-01,2016-04-15,9.28,6,55.68,1
455767,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,9353,2016-03-15,2016-03-25,2016-04-04,2016-04-30,9.28,6,55.68,1
733550,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,10090,2016-05-05,2016-05-12,2016-05-17,2016-07-03,9.28,5,46.40,1
826935,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,10234,2016-05-15,2016-05-24,2016-05-30,2016-07-01,9.28,6,55.68,1
1290622,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,11292,2016-07-20,2016-07-26,2016-08-01,2016-09-12,9.28,6,55.68,1
1341783,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,11336,2016-07-24,2016-08-04,2016-08-15,2016-09-19,9.28,6,55.68,1
1493888,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,11698,2016-08-17,2016-08-24,2016-09-01,2016-10-08,9.28,5,46.40,1
1593583,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,11895,2016-08-31,2016-09-07,2016-09-20,2016-10-25,9.28,6,55.68,1
